In [1]:
import pandas as pd
import altair as alt

In [2]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [3]:
ind_policy_df = pd.DataFrame({
    'year': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    'measure': [34, 20, 56, 144, 136, 242, 260, 228, 705, 632, 852, 1594, 1568, 2049],
    #'act/proj': ['Actual', 'Actual', 'Actual', 'Actual', 'Actual', 'Actual', 'Actual', 'Actual', 'Projected', 'Projected', 'Projected', 'Projected', 'Projected', 'Projected', 'Projected']  
})

In [4]:
base = alt.Chart(ind_policy_df).encode(
    x='year:O',
    y=alt.Y('measure:Q'),
    #color=alt.Color('act/proj:N', legend=alt.Legend(orient='top')),
    tooltip=[
    #alt.Tooltip('act/proj:N', title='Actual/Projected'),
    alt.Tooltip('year:O', title='Year'),
    alt.Tooltip('measure:Q', title='Industrial Policies', format=',.0f')
]
)

line_points = base.mark_line(point=True)

# Custom y-axis label as text, rotated and positioned manually
title_ = alt.Chart(pd.DataFrame({'label': ['Trend of new industrial policy measures, 2010–23']})).mark_text(
    angle=0,
    align='left',
    fontSize=15,
    fontWeight='bold',
    dy=-170,  # vertical offset (adjust as needed)
    dx=-205     # horizontal offset (adjust as needed)
).encode(
    text='label:N'
)

# Layer the fake y-axis label and the bar chart
chart1 = alt.layer(title_, line_points).resolve_scale(
    color='independent'
).configure_axis(
    labelFontSize=12
)

chart1

alt.LayerChart(...)

In [5]:
# Save to png
chart1.save('IPM_chart.png', scale_factor=2)
# Save to json
chart1.save('IPM_chart.json', scale_factor=2)

In [28]:
chart2_data = {
    "Policy instrument": [
        "Financial grant (AE)", "State loan (AE)", "State aid, unespecified (AE)",
        "Public procurement localisation (AE)", "Trade finance (AE)", "Financial assistance in foreign markets (AE)",
        "Controls on transactions and investments (AE)", "Loan guarantee (AE)", "Tax or social insurance relief (AE)",
        "Capital and equity injection (AE)", "Import tariff (EMDE)", "State loan (EMDE)",
        "Tax or social insurance relief (EMDE)", "Capital and quity injection (EMDE)", "State aid, unespecified (EMDE)",
        "Financial grant (EMDE)", "Export tax (EMDE)", "Public procurement localisation (EMDE)",
        "Import licensing requirement (EMDE)", "Internal taxation of imports (EMDE)"
    ],
    "Share in industrial policy (%)": [
        27, 12, 9, 7, 5, 5, 5, 4, 4, 3, 14, 11, 11, 6, 5, 5, 5, 4, 4, 4
    ],
    "Type": [
        "Subsidies", "Subsidies", "Subsidies", "Other calculations", "Subsidies", "Subsidies",
        "Other calculations", "Subsidies", "Subsidies", "Subsidies", "Import/Export measures",
        "Subsidies", "Subsidies", "Subsidies", "Subsidies", "Subsidies", "Import/Export measures",
        "Other calculations", "Import/Export measures", "Import/Export measures"
    ],
    "country": [
        "Advanced economies"] * 10 + ["Emerging market and developing economies"] * 10
}

IP_share_df = pd.DataFrame(chart2_data)


In [29]:
IP_share_df['Share in industrial policy (%)'] = IP_share_df['Share in industrial policy (%)'] / 100

In [30]:
color_mapping = {
    'Subsidies': '#179fdb',
    'Other calculations': '#f4c245',
    'Import/Export measures': '#e6224b'
}

In [31]:
custom_order = [
    "Financial grant (AE)", "State loan (AE)", "State aid, unespecified (AE)",
        "Public procurement localisation (AE)", "Trade finance (AE)", "Financial assistance in foreign markets (AE)",
        "Controls on transactions and investments (AE)", "Loan guarantee (AE)", "Tax or social insurance relief (AE)",
        "Capital and equity injection (AE)", "Import tariff (EMDE)", "State loan (EMDE)",
        "Tax or social insurance relief (EMDE)", "Capital and quity injection (EMDE)", "State aid, unespecified (EMDE)",
        "Financial grant (EMDE)", "Export tax (EMDE)", "Public procurement localisation (EMDE)",
        "Import licensing requirement (EMDE)", "Internal taxation of imports (EMDE)"
]

In [32]:
for x in range(len(IP_share_df)):
    IP_share_df.at[x, 'Policy instrument'] = IP_share_df.at[x, 'Policy instrument'].replace(' (AE)', '')
    IP_share_df.at[x, 'Policy instrument'] = IP_share_df.at[x, 'Policy instrument'].replace(' (EMDE)', '')

In [33]:
rows = []

for country_type in IP_share_df.country.unique():
    rows.append(alt.Chart(IP_share_df.query("country == @country_type")).encode(
        x=alt.X("Share in industrial policy (%)", axis=alt.Axis(format=".0%")),
        y=alt.Y("Policy instrument:N",sort=custom_order, axis=alt.Axis(
            title=None,
            labelLimit=400)),
        color=alt.Color("Type:N", legend=alt.Legend(orient='top'), scale=alt.Scale(
                            domain=list(color_mapping.keys()),
                            range=list(color_mapping.values())
                        )),
        tooltip=[
            alt.Tooltip('country:N', title='Economy'),
            alt.Tooltip('Policy instrument:N', title='Policy instrument'),
            alt.Tooltip('Share in industrial policy (%)', title='Share in industrial policies', format='.0%')
        ]
    ).mark_bar().properties(
        title=alt.TitleParams(
            text=country_type,
            fontSize=12,
            fontWeight='bold',
        ),
        height=300, width=300))

alt.vconcat(
    *rows,
)

alt.VConcatChart(...)

In [34]:
all_econs = alt.vconcat(
    *rows,
)

In [35]:
# Save to png
all_econs.save('share_of_IPs.png', scale_factor=2)
# Save to json
all_econs.save('share_of_IPs.json', scale_factor=2)

In [36]:
rows = []

for country_type in IP_share_df.country.unique():
    rows.append(alt.Chart(IP_share_df.query("country == @country_type")).encode(
        x=alt.X("Share in industrial policy (%)", axis=alt.Axis(format=".0%")),
        y=alt.Y("Policy instrument:N",sort=custom_order, axis=alt.Axis(
            title=None,
            labelLimit=400)),
        color=alt.Color("Type:N", legend=alt.Legend(orient='top'), scale=alt.Scale(
                            domain=list(color_mapping.keys()),
                            range=list(color_mapping.values())
                        )),
        tooltip=[
            alt.Tooltip('country:N', title='Economy'),
            alt.Tooltip('Policy instrument:N', title='Policy instrument'),
            alt.Tooltip('Share in industrial policy (%)', title='Share in industrial policies', format='.0%')
        ]
    ).mark_bar().properties(
        title=alt.TitleParams(
            text=country_type,
            fontSize=12,
            fontWeight='bold',
        ),
        height=300, width=300))

rows[0]

alt.Chart(...)

In [37]:
advanced_econs = rows[0]

In [38]:
# Save to png
advanced_econs.save('share_of_IPs_ae.png', scale_factor=2)
# Save to json
advanced_econs.save('share_of_IPs_ae.json', scale_factor=2)